<a href="https://colab.research.google.com/github/ikanx101/AppSheet-Sales/blob/main/AppSheet_JATIM_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# _AppSheet Converter_ 2025 untuk __JAWA TIMUR__

_Converter_ ini dibuat untuk mengkonversi langsung __tiga *sheets*__ yang ada pada data tarikan __AppSheet__ dari tim DD Jawa Timur. Proses kerjanya adalah dengan _merging_ dan _enriching_ data pada _sheets_: `Order`, `Call`, dan `Merchandise` untuk kemudian di-_convert_ ke dalam satu _output file_ berformat Excel.

__Cara pakai:__

1. Klik tombol `Connect` pada pojok kanan atas untuk menyalakan _server cloud Google Colab_.
1. _Upload_ _files_ yang dibutuhkan, yakni:
    - Data tarikan __AppSheet__.
        - Perhatikan bahwa penamaan _sheets_ yang hendak diproses (`Order`, `Call`, dan `Merchandise`) __tidak boleh berubah sama sekali__. Seandainya ada perubahan, silakan kontak saya untuk penyesuaian.
    - Data __*Master Item*__.
        - Perhatikan bahwa _database_ _master item_ berada di _sheet_ bernama `Product`.
1. Isi _form_ sesuai dengan nama _files_ tersebut.
1. Klik `Runtime` >> `Run all`.
1. Tunggu hingga proses selesai (sampai muncul pesan `PROSES SELESAI`). Kira-kira akan memakan waktu 5-15 menit.
1. Akan muncul _file_ baru bernama `Jatim_converted.xlsx`. Silakan di-_download_.

Catatan: Karena yang kita gunakan saat ini adalah _server_ gratisnya Google, jadi kita harus selalu _install_ beberapa _packages_ setiap kali _run_ skrip ini. Konsekuensinya _runtime_-nya lebih lama.

_Last Update_: 16 Juni 2025

In [ ]:
                #@title Masukkan Nama Files

rm(list=ls())
gc()

nama_file_utama  <- "01. Data NutriGO - Jan 2025.xlsx" #@param {type:"string"}
nama_file_master <- "Master Data.xlsx" #@param {type:"string"}

# nama sheet untuk di file utama
sht  = "Merchandise"

if(!require(readxl)){
  install.packages("readxl")
  library(readxl)
}
if(!require(reshape2)){
  install.packages("reshape2")
  library(reshape2)
}
if(!require(janitor)){
  install.packages("janitor")
  library(janitor)
}
if(!require(openxlsx)){
  install.packages("openxlsx")
  library(openxlsx)
}
if(!require(expss)){
  install.packages("expss")
  library(expss)
}

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,696463,37.2,1454458,77.7,1454458,77.7
Vcells,1313683,10.1,8388608,64.0,8308691,63.4


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘checkmate’, ‘maditr’, ‘htmlTable’, ‘matrixStats’




In [ ]:
# @title
# libraries yang dibutuhkan
library(dplyr)
library(tidyr)
library(openxlsx)
library(janitor)
library(readxl)
library(expss)

# ==============================================================================
# kita mengambil master data item
file = nama_file_master
sht  = "Product"
df_master =
  read_excel(file,sheet = sht) |>
  janitor::clean_names() |>
  select(item_group,category,brand) |>
  rename("Nama Item" = item_group,
         Brand       = brand,
         Category    = category)
# ==============================================================================

# ==============================================================================
# sekarang kita ambil data utamanya
file = nama_file_utama
sht  = c("Order","Call","Merchandise")

# kita ambil data order, call, dan merchandise
df_order =
  read_excel(file,sheet = sht[1]) %>%
  janitor::clean_names() %>%
  mutate(tanggal = as.Date(tanggal,"%Y-%m-%d"))
colnames(df_order) = read_excel(file,sheet = sht[1]) %>% colnames()

df_call =
  read_excel(file,sheet = sht[2]) %>%
  janitor::clean_names() %>%
  mutate(check_in  = as.POSIXlt(check_in, format="%Y-%m-%d %H:%M:%S")) %>%
  mutate(check_in  = format(check_in,"%H:%M:%S")) %>%
  mutate(check_out = as.POSIXlt(check_out, format="%Y-%m-%d %H:%M:%S")) %>%
  mutate(check_out = format(check_out,"%H:%M:%S")) %>%
  mutate(durasi    = as.POSIXlt(durasi, format="%Y-%m-%d %H:%M:%S")) %>%
  mutate(durasi    = format(durasi,"%H:%M:%S")) %>%
  mutate(tanggal   = as.Date(tanggal,"%Y-%m-%d"))
colnames(df_call) = read_excel(file,sheet = sht[2]) %>% colnames()

df_av =
  read_excel(file,sheet = sht[3]) %>%
  janitor::clean_names() %>%
  mutate(tanggal = as.Date(tanggal,"%Y-%m-%d"))
colnames(df_av) = read_excel(file,sheet = sht[3]) %>% colnames()
# ==============================================================================


# ==============================================================================
# sekarang kita akan ambil gabung call dan order
df_order$`Tipe Transaksi` = "Call"

df_call_order =
  df_call %>%
  select(`ID Call`,`Check In`,
         `Nama Cluster Firestart`,
         `Koordinat RO`,`Koordinat Call`,`Jarak (Meter)`,`Kesesuaian Titik`,
         `Check Out`,`Durasi`)

df_gabung = merge(df_order,df_call_order,by = "ID Call") %>% distinct()

df_gabung_1 =
  df_gabung %>%
  mutate(`Peserta Display Wow Operator` = NA,
         `Peserta Loyalty Sachet`       = NA,
         `Project OTG`                  = NA,
         `Nama Distributor`             = NA,
         `Kecamatan Distributor`        = NA,
         `Project 1`                    = NA,
         `Project 2`                    = NA,
         `Status AV`                    = NA,
         `Berat(Gram)`                  = NA,
         `Firestart NS`                 = NA,
         `Firestart Hilo`               = NA
         ) %>%
  mutate(`Status Sekolah` = ifelse(Sekolah == "Bukan Sekolah",FALSE,TRUE)) %>%
  select(`ID Order`,`ID Call`,`Check In`,`Tanggal`,`Bulan`,`ID MDS`,`Nama MDS`,
         `PIC`,`Area MDS`,`Region MDS`,`Kode Customer`,`Nama Customer`,
         `Kecamatan`,`Kabupaten`,`Provinsi`,`Alamat`,`Detail Klasifikasi`,
         `Klasifikasi`,`Tipe Customer`,`Sekolah`,`Nama Cluster Firestart`,
         `Koordinat RO`,`Koordinat Call`,`Jarak (Meter)`,`Kesesuaian Titik`,
         `Peserta Display Wow Operator`,`Peserta Loyalty Sachet`,`Project OTG`,
         `Nama Distributor`,`Kecamatan Distributor`,`Project 1`,`Project 2`,
         `Check Out`,`Durasi`,`Brand`,`Category`,`Nama Item`,`Qty`,`Value`,
         `Status AV`,`Berat(Gram)`,`Tipe Transaksi`,`Firestart NS`,`Firestart Hilo`,
         `Status Sekolah`)
# ==============================================================================


# ==============================================================================
# sekarang kita akan gabung av dan merchandise
df_av_merch =
  df_av %>%
  select(`ID Merch`,`ID Call`,`Tanggal`,`Bulan`,`ID MDS`,`Nama MDS`,`PIC`,
         `Area MDS`,`Region MDS`,`Kode Customer`,`Nama Customer`,`Kecamatan`,
         `Kabupaten`,`Provinsi`,`Alamat`,`Detail Klasifikasi`,`Klasifikasi`,
         `Tipe Customer`,`Sekolah`) %>%
  rename(`ID Order` = `ID Merch`) %>%
  mutate(`Tipe Transaksi` = "AV") %>%
  mutate(`Peserta Display Wow Operator` = NA,
         `Peserta Loyalty Sachet`       = NA,
         `Project OTG`                  = NA,
         `Nama Distributor`             = NA,
         `Kecamatan Distributor`        = NA,
         `Project 1`                    = NA,
         `Project 2`                    = NA,
         `Status AV`                    = NA,
         `Berat(Gram)`                  = NA,
         `Firestart NS`                 = NA,
         `Firestart Hilo`               = NA) %>%
  mutate(`Status Sekolah` = ifelse(Sekolah == "Bukan Sekolah",FALSE,TRUE))

batas       = "Tier Lokalate & Wdank"
batas_hapus = 2:which(colnames(df_av) == batas)

df_av_produk =
  df_av[-batas_hapus] %>%
  rename(`ID Order` = `ID Merch`) %>%
  melt(id.vars = "ID Order",na.rm = T) %>%
  filter(value > 0) %>%
  rename("Nama Item" = variable) %>%
  rename(Qty = value) %>%
  merge(df_master) %>%
  mutate(Value = NA)

df_av_order =
  df_call %>%
  select(`ID Call`,`Check In`,
         `Nama Cluster Firestart`,
         `Koordinat RO`,`Koordinat Call`,`Jarak (Meter)`,`Kesesuaian Titik`,
         `Check Out`,`Durasi`)

df_gabung_order_1 = merge(df_av_merch,df_av_order,by = "ID Call") %>% distinct()
df_gabung_2       = merge(df_gabung_order_1,df_av_produk,by = "ID Order") %>% distinct()

df_gabung_2 =
  df_gabung_2 %>%
  select(`ID Order`,`ID Call`,`Check In`,`Tanggal`,`Bulan`,`ID MDS`,`Nama MDS`,
         `PIC`,`Area MDS`,`Region MDS`,`Kode Customer`,`Nama Customer`,
         `Kecamatan`,`Kabupaten`,`Provinsi`,`Alamat`,`Detail Klasifikasi`,
         `Klasifikasi`,`Tipe Customer`,`Sekolah`,`Nama Cluster Firestart`,
         `Koordinat RO`,`Koordinat Call`,`Jarak (Meter)`,`Kesesuaian Titik`,
         `Peserta Display Wow Operator`,`Peserta Loyalty Sachet`,`Project OTG`,
         `Nama Distributor`,`Kecamatan Distributor`,`Project 1`,`Project 2`,
         `Check Out`,`Durasi`,`Brand`,`Category`,`Nama Item`,`Qty`,`Value`,
         `Status AV`,`Berat(Gram)`,`Tipe Transaksi`,`Firestart NS`,`Firestart Hilo`,
         `Status Sekolah`)

df_final = rbind(df_gabung_1,df_gabung_2)

# sekarang kita akan pecah ya
marker = nrow(df_final)
batas_pisah = 10^6

if(marker < batas_pisah){
  output = list(df_final)
}
if(marker > batas_pisah){
  output_1 = df_final[1:batas_pisah,]
  output_2 = df_final[(batas_pisah + 1):marker,]
  output   = list(output_1,output_2)
}

data_jatim = output
wb <- createWorkbook()

for(ikanx in 1:length(data_jatim)){
  sh = addWorksheet(wb, paste0("Sheet ",ikanx))
  xl_write(data_jatim[ikanx], wb, sh)
}

# Menyimpan workbook ke file
saveWorkbook(wb, "Jatim_converted.xlsx",overwrite = T)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test


Loading required package: maditr


To aggregate data: take(mtcars, mean_mpg = mean(mpg), by = am)



Attaching package: ‘maditr’


The following objects are masked from ‘package:dplyr’:

    between, coalesce, first, last



Attaching package: ‘expss’


The following objects are masked from ‘package:tidyr’:

    contains, nest


The following objects are masked from ‘package:dplyr’:

    compute, contains, na_if, recode, vars, where


Warning message in data.table::melt.data.table(data.table::as.data.table(.), id.vars = "ID Order", :
“'measure.vars' [NS ANGGUR HIJAU PLS 4PX40SX11G, NS ANGGUR PLS 4PX40SX11G, NS APEL JERUK PLS 4PX40SX11G, NS ASO PLS 18PX40SX14G, ...] are not

In [ ]:
#@title Status
print("PROSES SELESAI")
print("file hasil konversi tersimpan dalam nama: Jatim_converted.xlsx")

[1] "PROSES SELESAI"
[1] "file hasil konversi tersimpan dalam nama: Call Converted.xlsx"
